In [1]:
import json
import datetime


In [5]:
# у меня ноут не выдерживает эту ячеечку из юпитера, но ваш точно должен справиться. 
!../build/tgnews json ../data --lang_detect_model ../models/lang_detect.ftz --cat_detect_model ../models/cat_detect.ftz --news_detect_model ../models/news_detect.ftz --languages ru > parsed_docs_ru.json

Mode: json
Loading models...
FastText lang_detect model loaded
Reading file names...
Files count: 500000
Parsing 500000 files...
262698 documents saved
std::bad_alloc


In [3]:
with open("parsed_docs_ru.json") as f:
    articles = json.load(f)
articles[0]

{'date': '2019-11-20T09:17:00+00:00',
 'description': 'В рамках федерального проекта «Формирование комфортной городской среды» национального проекта «Жилье и городская среда» в субъектах Северо-Западного и Сибирского федеральных округов в текущем году будет благоустроено 3857 объектов, работы еще продолжаются на 386 территориях и должны быть завершены до конца года. Об этом сообщил Максим Егоров, заместитель главы Минстроя России, на совещаниях, организованных полномочными представителями президента в федеральных округах 18 и 19 ноября.',
 'site_name': 'Глас Народа',
 'text': 'В рамках федерального проекта «Формирование комфортной городской среды» национального проекта «Жилье и городская среда» в субъектах Северо-Западного и Сибирского федеральных округов в текущем году будет благоустроено 3857 объектов, работы еще продолжаются на 386 территориях и должны быть завершены до конца года. Об этом сообщил Максим Егоров, заместитель главы Минстроя России, на совещаниях, организованных полном

In [4]:
for article in articles:

    article_date = article["date"][:-3] + article["date"][-2:]
    if article_date: # some of articles have no date
        article_datetime = datetime.datetime.strptime(article_date, '%Y-%m-%dT%H:%M:%S%z')
        article["ts"] = article_datetime.timestamp()
    else:
        article["ts"] = 0.0
    
    del article["date"]


In [5]:
articles = sorted(articles, key=lambda x: x["ts"])

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer as tf_idf_vectorizer

TRAIN_SIZE = 0.8
articles_train, articles_test = articles[:int(TRAIN_SIZE * len(articles))], articles[int(TRAIN_SIZE * len(articles)):]

In [105]:
def get_title_text(x):
    return x["title"] + " " + x["text"]

tfidf = tf_idf_vectorizer(max_df=0.1, min_df=0.001)
X = tfidf.fit_transform(map(get_title_text, articles_train))

In [106]:
X.shape

(163169, 18942)

In [107]:
X_test = tfidf.transform(map(get_title_text, articles_test))

In [108]:
X_test.shape

(40793, 18942)

In [171]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.7, min_samples=2).fit(X_test)
len(set(clustering.labels_))

2329

In [172]:
for i in range(X_test.shape[0]):
    articles_test[i]["cluster"] = clustering.labels_[i]


In [173]:
list(map(lambda x: x["title"], list(filter(lambda x: x["cluster"] == 0, articles_test))))

['Разумков об общении с Богданом: Бывают жесткие дискуссии',
 'Разумков рассказал о "жестких дискуссиях" с Богданом',
 'Разумков признался, что часто дискутирует с Богданом',
 '«Дискутируем мы»: Разумков рассказал о спорах с Богданом']